# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy


# Import API key
from api_keys import g_key



### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data = pd.read_csv("output_data/cities.csv")
weather_data

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed(mph)
0,Port Alfred,8,ZA,1613633446,78,-33.5906,26.8910,75.00,7.00
1,San Patricio,90,US,1613633446,100,28.0170,-97.5169,35.60,14.97
2,Kaitangata,98,NZ,1613633446,74,-46.2817,169.8464,61.00,1.99
3,Cedar City,1,US,1613633447,73,37.6775,-113.0619,23.00,3.44
4,San Isidro,0,PE,1613633447,83,-12.1167,-77.0500,71.01,4.61
...,...,...,...,...,...,...,...,...,...
573,Bandarbeyla,99,SO,1613633575,55,9.4942,50.8122,78.75,11.07
574,Marsassoum,96,SN,1613633575,64,12.8275,-15.9806,75.20,4.61
575,Hobyo,100,SO,1613633575,63,5.3505,48.5268,80.80,14.07
576,Pāsighāt,44,IN,1613633576,24,28.0667,95.3333,84.49,3.29


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
 #Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = weather_data[["Lat", "Lng"]]

# Store Humidity in humidity
humidity = weather_data["Humidity"]

In [4]:
 # Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

In [5]:
m=gmaps.figure()
m.add_layer(gmaps.heatmap_layer(locations,weights=humidity))
m


Figure(layout=FigureLayout(height='420px'))

In [6]:
max_intensity

100

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
# Narrow down the cities with wind speed less than 10 mph, cloudiness equals to 0 and max temp between 60 and 80
narrowed_city_df = weather_data.loc[(weather_data["Wind Speed(mph)"] <= 10) & (weather_data["Cloudiness"] == 0) & \
                                   (weather_data["Max Temp"] >= 70) & (weather_data["Max Temp"] <= 80)].dropna()

narrowed_city_df=narrowed_city_df[:20]
narrowed_city_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed(mph)
4,San Isidro,0,PE,1613633447,83,-12.1167,-77.0500,71.01,4.61
6,Zhuhai,0,CN,1613633447,49,22.2769,113.5678,75.99,4.61
34,Paita,0,PE,1613633454,79,-5.0892,-81.1144,73.51,6.91
204,San Juan,0,AR,1613633493,54,-31.5375,-68.5364,76.28,7.72
221,Hong Kong,0,HK,1613633282,58,22.2855,114.1577,70.00,1.99
258,Lakes Entrance,0,AU,1613633505,69,-37.8811,147.9810,75.00,5.99
271,Vila Velha,0,BR,1613633467,78,-20.3297,-40.2925,77.00,9.22
345,San Miguel de Tucumán,0,AR,1613633524,83,-26.8241,-65.2226,73.40,3.15
365,Lima,0,PE,1613633528,83,-12.0432,-77.0282,71.01,4.61
393,Gerāsh,0,IR,1613633415,23,27.6652,54.1371,71.60,7.92


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
# Create a hotel_df
hotel_df = narrowed_city_df.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

,City,Country,Lat,Lng,Hotel Name
4,San Isidro,PE,-12.1167,-77.0500,
6,Zhuhai,CN,22.2769,113.5678,
34,Paita,PE,-5.0892,-81.1144,
204,San Juan,AR,-31.5375,-68.5364,
221,Hong Kong,HK,22.2855,114.1577,
258,Lakes Entrance,AU,-37.8811,147.9810,
271,Vila Velha,BR,-20.3297,-40.2925,
345,San Miguel de Tucumán,AR,-26.8241,-65.2226,
365,Lima,PE,-12.0432,-77.0282,
393,Gerāsh,IR,27.6652,54.1371,


In [9]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [10]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
    

# Print end of search once searching is completed
print("-------End of Search-------")

Retrieving Results for Index 4: San Isidro.
Closest hotel in San Isidro is Hampton by Hilton Lima San Isidro.
------------
Retrieving Results for Index 6: Zhuhai.
Closest hotel in Zhuhai is Pullman Zhuhai.
------------
Retrieving Results for Index 34: Paita.
Closest hotel in Paita is Hotel Marina Del Bay.
------------
Retrieving Results for Index 204: San Juan.
Closest hotel in San Juan is Hotel Viñas del Sol.
------------
Retrieving Results for Index 221: Hong Kong.
Closest hotel in Hong Kong is The OTTO Hotel.
------------
Retrieving Results for Index 258: Lakes Entrance.
Closest hotel in Lakes Entrance is The Esplanade Resort & Spa.
------------
Retrieving Results for Index 271: Vila Velha.
Closest hotel in Vila Velha is Hotel Senac Ilha do Boi.
------------
Retrieving Results for Index 345: San Miguel de Tucumán.
Closest hotel in San Miguel de Tucumán is Sheraton Tucuman Hotel.
------------
Retrieving Results for Index 365: Lima.
Closest hotel in Lima is Hotel Carrera.
------------

In [14]:
# Display the hotel dataframe
hotel_df

,City,Country,Lat,Lng,Hotel Name
4,San Isidro,PE,-12.1167,-77.0500,Hampton by Hilton Lima San Isidro
6,Zhuhai,CN,22.2769,113.5678,Pullman Zhuhai
34,Paita,PE,-5.0892,-81.1144,Hotel Marina Del Bay
204,San Juan,AR,-31.5375,-68.5364,Hotel Viñas del Sol
221,Hong Kong,HK,22.2855,114.1577,The OTTO Hotel
258,Lakes Entrance,AU,-37.8811,147.9810,The Esplanade Resort & Spa
271,Vila Velha,BR,-20.3297,-40.2925,Hotel Senac Ilha do Boi
345,San Miguel de Tucumán,AR,-26.8241,-65.2226,Sheraton Tucuman Hotel
365,Lima,PE,-12.0432,-77.0282,Hotel Carrera
393,Gerāsh,IR,27.6652,54.1371,Chelpasin


In [15]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# Add marker layer and info box content ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))